**Qualitative Analysis of Validation Responses**

In [12]:
import json
import pandas as pd
import os


model_name = "gpt-4o-2024-05-13" # gpt-4o-2024-05-13 gpt-3.5-turbo-0125 llama3:8b llama3:70b
data_file = f"../data/evaluation/without/all_dependencies_without_{model_name}.json"
baseline_file = "../data/evaluation/all_dependencies.csv"
output_dir = "../data/evaluation/analysis"
output_file = os.path.join(output_dir, "without_" + model_name + ".csv")


with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)

df = pd.read_csv(baseline_file)


errors = []


for entry, (index, row) in zip(data, df.iterrows()):
    
    if "responses" in entry:
        response = entry["responses"][0]
    else:
        response = entry["response"]

    try:
        if "responses" in entry:
            response = entry["responses"][0]
        else:
            response = entry["response"]
        response_dict = json.loads(response)
        isDependency = response_dict["isDependency"]
    except json.JSONDecodeError as error:
        print(error)

    rating = row["final_rating"]
    


    # checl borderline cases
    if str(rating) == "Borderline":
        continue
        #errors.append({
        #    "index": index,
        #    "model_name": model_name,
        #    "rating": "borderline",
        #    "response_rating": str(isDependency),
        #    "term": "None"
        #})

    
    # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
    if isDependency and str(rating).lower() == "false":
        errors.append({
            "index": index,
            "model_name": model_name,
            "rating": rating,
            "response_rating": str(isDependency),
            "option_types" 
            "term": "False Postive"
        })

    # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
    if not isDependency and  str(rating).lower() == "true":
        errors.append({
            "index": index,
            "model_name": model_name,
            "rating": rating,
            "response_rating": str(isDependency),
            "term": "False Postive"
        })



df_errors = pd.DataFrame(errors)   
df_errors.to_csv(output_file, index=False)

,index,model_name,rating,response_rating,term
0,0,gpt-4o-2024-05-13,True,False,False Postive
1,4,gpt-4o-2024-05-13,False,True,False Postive
2,5,gpt-4o-2024-05-13,False,True,False Postive
3,9,gpt-4o-2024-05-13,True,False,False Postive
4,10,gpt-4o-2024-05-13,False,True,False Postive
...,...,...,...,...,...
98,486,gpt-4o-2024-05-13,TRUE,False,False Postive
99,487,gpt-4o-2024-05-13,TRUE,False,False Postive
100,491,gpt-4o-2024-05-13,TRUE,False,False Postive
101,494,gpt-4o-2024-05-13,TRUE,False,False Postive
